In [4]:
import requests
import json

In [5]:
path = 'D:/WebShop/baseline_models/data/il_trajs_finalized_images/il_trajs_finalized_images.jsonl'
out_file = 'D:/WebShop/baseline_models/data/trajs_without_images.jsonl'
with open(path, 'r', encoding='utf-8') as f, open(out_file, 'w', encoding='utf-8') as o:
    for line in f:
        record = json.loads(line)
        record.pop("images",None)
        o.write(json.dumps(record, ensure_ascii=False)+"\n")

In [6]:
system_prompt = """You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-08-28\n\nReasoning: None\n\n# Valid channels: final. Channel must be included for every message.\nNever output analysis or commentary; respond only in the final channel."""

In [7]:
webshop_info = """You will be working on augmenting the dataset for human trajectories on Webshop.
Webshop is a simulated e-commerce website developed for training agents on RL tasks. It has a instruction/goal on which humans and agents act. Based on the instruction, a final product is purchased and a reward is given. The given reward will be 1, when the goal's categories, price constraints, and features match exactly with that of the product bought at the end of the trajectory.
You'll be given the human trajectory in the following format, {"step":"step-number","goal":"instruction for which human is working on","state": "the current state of the webpage, dom element text etc","actions_available": "all the actions that are available, like click[an element],search[something] etc","history":"list of all the actions taken up until that step.","action_chosen": "The actual action chosen by the human, we assume that this action is taken based on the history & current state.","action_meaning": "what the action means if we put the correct mmid in the action","next_state": "The state of the agent, i.e. webpage after the action is taken"}.
 For the json provided, you will have to generate 4 actions 1. plan_action, 2. thought_action, 3.env_action, 4.explantation_action. "plan_action" represents the plan to achieve the goal, the plan action keeps changing on the basis of current "state". It is to help the agent take next set of actions. Generate this by looking at current state, and the history provided to you. "thought_action" is kind of like a reasoning step, you need to look at the action chosen by human and arrive at their thought process, looking at the current state, history and plan, and present it in this field. Speak in the future tense as if you were thinking to take the action already taken by the human. This is you guessing why human took that action based on the value in the "action_meaning" field in the input from human's perspective. "env_action" the action taken by the user to interact with the RL environment, it's just "action_meaning" in the input. "expl_action", looking at the current_state, history, action_taken, and the next_state agent went into, you need to explain what happened because we took that action. Note that you need to generate all these 4 fields for already existing trajectories by Humans. If you see click[buy now], that means it's the end of the trajectory. The first state will always be a blank page with just an instruction and a search bar in it. Generally humans search for something and if the results are not up to the mark, they may return to the home page. If we are on homepage, the action space is always empty. You can assume, we must always search some keywords. You should always speak in the first person in all the action generations. Keep the length of various actions to less than 30 words each.
 The output should be a valid JSON with exactly 4 keys and 4 values as shown below. Expected output format: {"plan": "<plan_action>","thought":"<thought_action>","env":"<env_action>","expl": "<expl_action>"}."""

In [8]:
system_prompt = system_prompt + webshop_info

In [9]:
url = "http://localhost:8001/v1/chat/completions"
headers = {"Content-Type": "application/json"}

def convert_to_agent_q(step_prompt):
    payload = {
        "model": "gpt-oss-20b",  # or whatever you named the model
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": str(step_prompt)}
        ],
        "max_tokens": 40000,
        "temperature": 0.7,
        "response_format": {"type":"json_object"},
        "stream": False  # this ensures you get a JSON-only response instead of an SSE stream
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    print(response.json())
    if response.status_code==200:
        return response.json()['choices'][0]['message']['content']
    else:
        return None

In [10]:
def prepare_dataset(current_record):
    record_value = []
    actions = current_record['actions']
    states = current_record['states']
    available_actions = current_record['available_actions']
    actions_translate = current_record['actions_translate']
    instruction = current_record['states'][0].split('Instruction: \n')[1].split('\n')[0]
    i=0
    for i in range(len(actions)-1):
        step = {'step': i + 1,'goal': instruction, 'state': states[i], 'actions_available': available_actions[i], 'history': actions_translate[:i], 'action_chosen': actions[i], 'action_meaning': actions_translate[i],'next_state': states[i+1]}
        action_op = convert_to_agent_q(step)
        record_value.append({'input':step,'output':action_op})
    i=i+1
    step = {
        'step': i+1,
        'goal': instruction,
        'state': states[i],
        'actions_available': available_actions[i],
        'action_chosen': actions[i],
        'action_meaning': actions_translate[i],
        'next_state': "GOAL_COMPLETED, TERMINAL_STATE, REWARD_FOUND."
    }
    action_op = convert_to_agent_q(step)
    record_value.append({'input':'','output':action_op})
    return record_value

In [11]:
record = {}
human_benchmark_file = "D:/WebShop/baseline_models/data/grpo.jsonl"
with open(out_file, 'r', encoding='utf-8') as f, open(human_benchmark_file, 'w', encoding='utf-8') as o:
    for line in f:
        record = json.loads(line)
        output = prepare_dataset(record)
        #print(output)
        o.write(json.dumps(output, ensure_ascii=False)+"\n")
        break

{'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '{"plan":"I will browse the results for a 32\u202fGB high‑speed USB, filter by price under $50, then select the best match.","thought":"I need a fast 32\u202fGB flash drive for under $50, so I will look at the first page of results and choose the cheapest high‑speed option.","env":"search[32 gb high speed usb flash drive ]","expl":"The search returns many 32\u202fGB USB drives; the page shows several priced under $50 and offers options to go next."}'}}], 'created': 1756417221, 'model': 'gpt-oss-20b', 'system_fingerprint': 'b6301-da54f9f1', 'object': 'chat.completion', 'usage': {'completion_tokens': 111, 'prompt_tokens': 1549, 'total_tokens': 1660}, 'id': 'chatcmpl-q5Nx95jLFDE2WS8e2JEB2eFQc7aVyDoA', 'timings': {'prompt_n': 1549, 'prompt_ms': 6050.257, 'prompt_per_token_ms': 3.905911555842479, 'prompt_per_second': 256.02218219821077, 'predicted_n': 111, 'predicted_ms': 5436.349, 'predicted_per_

KeyboardInterrupt: 